In [ ]:
# Day 6 — Intro to SQL (SQLite) in Jupyter
**Goals:** Practice `SELECT`, `WHERE`, `ORDER BY` on an environmental dataset.

In [1]:
#Standard imports
import sqlite3
from pathlib import Path
import pandas as pd

#Create (or open) a local SQLite database file in your repo/notebook folder
db_path = Path("env_day6.sqlite")
conn = sqlite3.connect(db_path)

#For nicer DataFrame display
pd.set_option("display.precision", 3)

print(f"Connected to SQLite at {db_path.resolve()}")

Connected to SQLite at C:\Users\chamb\env-data-analyst-oct\notebooks\env_day6.sqlite


In [2]:
#Build a tiny environmental dataset

#Minimal water quality dataset (NO3_N = nitrate as N)

data = [
    # sample_id, site,            date,        parameter, value, units
    (1,  "Willow Creek",    "2025-08-12", "NO3_N",   2.3,  "mg/L"),
    (2,  "Willow Creek",    "2025-09-02", "NO3_N",   1.8,  "mg/L"),
    (3,  "Willow Creek",    "2025-09-28", "NO3_N",   2.9,  "mg/L"),
    (4,  "Bear River",      "2025-08-15", "NO3_N",   0.9,  "mg/L"),
    (5,  "Bear River",      "2025-09-04", "NO3_N",   1.2,  "mg/L"),
    (6,  "Bear River",      "2025-09-20", "NO3_N",   0.7,  "mg/L"),
    (7,  "Columbia Slough", "2025-08-10", "NO3_N",   3.1,  "mg/L"),
    (8,  "Columbia Slough", "2025-08-30", "NO3_N",   2.7,  "mg/L"),
    (9,  "Columbia Slough", "2025-09-21", "NO3_N",   3.8,  "mg/L"),

    # Add a second parameter (TP = total phosphorus) to make WHERE filters interesting
    (10, "Willow Creek",    "2025-08-12", "TP",      0.08, "mg/L"),
    (11, "Willow Creek",    "2025-09-02", "TP",      0.05, "mg/L"),
    (12, "Bear River",      "2025-08-15", "TP",      0.03, "mg/L"),
    (13, "Bear River",      "2025-09-20", "TP",      0.04, "mg/L"),
    (14, "Columbia Slough", "2025-08-10", "TP",      0.11, "mg/L"),
    (15, "Columbia Slough", "2025-09-21", "TP",      0.09, "mg/L"),
]

cols = ["sample_id", "site", "date", "parameter", "value", "units"]
df = pd.DataFrame(data, columns=cols)
df.head()

,sample_id,site,date,parameter,value,units
0,1,Willow Creek,2025-08-12,NO3_N,2.3,mg/L
1,2,Willow Creek,2025-09-02,NO3_N,1.8,mg/L
2,3,Willow Creek,2025-09-28,NO3_N,2.9,mg/L
3,4,Bear River,2025-08-15,NO3_N,0.9,mg/L
4,5,Bear River,2025-09-04,NO3_N,1.2,mg/L


In [ ]:
#Write the DataFrame into SQLite

#Write to SQLite(replace if already exists)
df.to_sql("water_quality", conn, if_exists="replace", index=False)  #Creates a "water_quality" table we can query with SQL

#Quick row count sanity check
pd.read_sql_query("SELECT COUNT(*) AS m_rows FROM water_quality;", conn)

,m_rows
0,15


In [ ]:
#Helper: run_sql() to keep results tidy
def run_sql(sql: str, params: dict | tuple | None = None):  #takes an SQL query, runs that query in SQLite DB you connect to, and returns results as Pandas DataFrame (which displays nicely in Jupyter NB)
            #sql:str=query passed as string(see below), params:... optional extra data(like filters passed into the query): can be a dict (key-value pair), a tuple(stores values in order, not by key), None means if SQL query doesnt need parameters then skip it
    """                                                     
    Execute a SQL query and return a pretty DataFrame.
    Use name (e.g. :param) or positional (?) parameters via `param`
    """
    return pd.read_sql_query(sql, conn, params=params)

#Tiny smoke test
run_sql("SELECT * FROM water_quality LIMIT 5")

,sample_id,site,date,parameter,value,units
0,1,Willow Creek,2025-08-12,NO3_N,2.3,mg/L
1,2,Willow Creek,2025-09-02,NO3_N,1.8,mg/L
2,3,Willow Creek,2025-09-28,NO3_N,2.9,mg/L
3,4,Bear River,2025-08-15,NO3_N,0.9,mg/L
4,5,Bear River,2025-09-04,NO3_N,1.2,mg/L


In [ ]:
#Practice: SELECT
#1 Everything (preview with LIMIT)
run_sql("""
SELECT *
FROM water_quality
LIMIT 5;
""")


,site,date,parameter,value,value_ug_L
0,Willow Creek,2025-08-12,TP,0.08,80.0
1,Willow Creek,2025-09-02,TP,0.05,50.0
2,Bear River,2025-08-15,TP,0.03,30.0
3,Bear River,2025-09-20,TP,0.04,40.0
4,Columbia Slough,2025-08-10,TP,0.11,110.0


In [ ]:
#Practice: SELECT
#2 Specify columns + alias
run_sql("""
SELECT
    sample_id,
    site,
    date,
    parameter AS param,
    value   AS result_mg_L
FROM water_quality
LIMIT 5;        
""")

,sample_id,site,date,param,result_mg_L
0,1,Willow Creek,2025-08-12,NO3_N,2.3
1,2,Willow Creek,2025-09-02,NO3_N,1.8
2,3,Willow Creek,2025-09-28,NO3_N,2.9
3,4,Bear River,2025-08-15,NO3_N,0.9
4,5,Bear River,2025-09-04,NO3_N,1.2


In [ ]:
#Practice: SELECT
#3 Simple expression (conver mg/L -> ug/L by x1000) when parameter is TP
run_sql("""
SELECT
    site,
    date,
    parameter,
    value,
    CASE
        WHEN parameter = 'TP' THEN value * 1000.0
        ELSE NULL
    END AS value_ug_L
FROM water_quality
WHERE parameter = 'TP'
LIMIT 5;        
""")

In [ ]:
#WHERE with comparisons, AND/OR, IN, BETWEEN, LIKE

#1 Filter by single equality
run_sql("""
SELECT site, date, parameter, value
FROM water_quality
WHERE site = :site AND parameter = 'NO3_N'
ORDER BY date;
""", params={"site": "Bear River"})

,site,date,parameter,value
0,Bear River,2025-08-15,NO3_N,0.9
1,Bear River,2025-09-04,NO3_N,1.2
2,Bear River,2025-09-20,NO3_N,0.7


In [14]:
#WHERE with comparisons, AND/OR, IN, BETWEEN, LIKE
#2 Numeric comparison (>), with AND ordered by value DESC
run_sql("""
SELECT site, date, parameter, value
FROM water_quality
WHERE parameter = 'NO3_N' AND value > :min_val
ORDER BY value DESC;
""", params={"min_val": 2.0})

,site,date,parameter,value
0,Columbia Slough,2025-09-21,NO3_N,3.8
1,Columbia Slough,2025-08-10,NO3_N,3.1
2,Willow Creek,2025-09-28,NO3_N,2.9
3,Columbia Slough,2025-08-30,NO3_N,2.7
4,Willow Creek,2025-08-12,NO3_N,2.3


In [15]:
#WHERE with comparisons, AND/OR, IN, BETWEEN, LIKE
#3 BETWEEN on ISO dates (Lexicographic works for 'YYYY-MM-DD')
run_sql("""
SELECT site, date, parameter, value
        FROM water_quality
        WHERE parameter = 'NO3_N'
            AND date BETWEEN :start AND :end
        ORDER BY date;
""", params={"start": "2025-08-01", "end": "2025-09-15"})

,site,date,parameter,value
0,Columbia Slough,2025-08-10,NO3_N,3.1
1,Willow Creek,2025-08-12,NO3_N,2.3
2,Bear River,2025-08-15,NO3_N,0.9
3,Columbia Slough,2025-08-30,NO3_N,2.7
4,Willow Creek,2025-09-02,NO3_N,1.8
5,Bear River,2025-09-04,NO3_N,1.2


In [16]:
#WHERE with comparisons, AND/OR, IN, BETWEEN, LIKE
#4 IN list (multiple sites)
run_sql("""
SELECT site, date, parameter, value
FROM water_quality
WHERE parameter = 'TP'
    AND site IN (:s1, :s2)
ORDER BY site, date;
""", params={"s1":"Willow Creek", "s2": "Columbia Slough"})

,site,date,parameter,value
0,Columbia Slough,2025-08-10,TP,0.11
1,Columbia Slough,2025-09-21,TP,0.09
2,Willow Creek,2025-08-12,TP,0.08
3,Willow Creek,2025-09-02,TP,0.05


In [17]:
#WHERE with comparisons, AND/OR, IN, BETWEEN, LIKE
#5 LIKE pattern (string match) - % = wildcard
run_sql("""
SELECT site, date, parameter, value
        FROM water_quality
        WHERE site LIKE :pat
        AND parameter = 'NO3_N'
        ORDER BY site, date;
""", params={"pat": "%Slough"})

,site,date,parameter,value
0,Columbia Slough,2025-08-10,NO3_N,3.1
1,Columbia Slough,2025-08-30,NO3_N,2.7
2,Columbia Slough,2025-09-21,NO3_N,3.8
